In [1]:
import pickle
import pandas as pd
import numpy as np
from load_data_old import *
from beam_search_module import *

In [2]:
stock_df = make_growth_target_df('datasets/stock_data_for_emm.pkl')
stock_df_index = stock_df.copy()
#stock_df.drop(['index'], inplace=True, axis=1)


In [3]:
stock_df_index

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,fullTimeEmployees,growth_target
0,France,Aerospace & Defense,EUR,Europe/Paris,PAR,Industrials,382.0,11.822,5.750674e+07,50.783,1102.0,"[0.0, 1.17, 2.41, 0.41, 2.65, -1.39, -7.24, -4..."
1,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,32.492,650.0,"[0.0, -1.84, 17.01, 3.7, -8.24, 8.01, -3.6, 4...."
2,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,181.181,34.0,"[0.0, -7.96, -2.61, 0.89, 13.94, -4.85, -6.12,..."
3,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,68.513,232.0,"[0.0, -3.39, 0.58, -10.17, 11.33, -2.91, 2.1, ..."
4,United Kingdom,Other Precious Metals & Mining,EUR,Europe/Berlin,FRA,Basic Materials,108.0,1.402,2.123667e+08,81.212,3474.0,"[0.0, 8.5, -15.79, 5.28, 2.24, -1.94, 7.5, -1...."
...,...,...,...,...,...,...,...,...,...,...,...,...
9774,Japan,Electrical Equipment & Parts,JPY,Europe/London,LSE,Industrials,4893.0,6.287,2.079737e+10,10.654,145696.0,"[0.0, 1.08, 10.44, -11.75, -11.15, 7.96, -9.17..."
9775,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,78360.0,"[0.0, 5.82, 1.71, 7.84, -10.55, 3.58, -7.94, 2..."
9776,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,4894.0,"[0.0, 6.5, 0.0, -10.85, -1.95, -3.62, -3.95, 9..."
9777,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,1.943,1297.0,"[0.0, 0.0, 0.0, 0.0, 18.86, 0.0, 0.0, 0.0, 10...."


In [4]:
data = stock_df.values.tolist()
column_names = list(stock_df.columns)
beam_width = 10
beam_depth = 3
nr_bins = 8
nr_saved = 10
subgroup_size = len(data) * 0.05
target = 'growth_target'
window_size = 5
target_ind = column_names.index(target)
all_time_series = [i[target_ind] for i in data]
all_time_series = np.array(all_time_series)
targets_baseline = np.mean(all_time_series, axis=0)
att_indices = list(range(0, len(column_names)))
att_indices.remove(target_ind)
att_columns = [column_names[i] for i in att_indices]
types = categorize_columns_in_order(stock_df, att_columns)
min_quality_improv = 0.25

In [5]:
results = beam_search_with_constraint(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved,
                      subgroup_size, target, types, window_size, min_quality_improv)


In [6]:
for r in results:
    print(r)
    print('\n')

(7.274882219354637, (('country', 'Japan', <function eq at 0x000001A5D43F9E10>), ('enterpriseToEbitda', -1.044, <function gt at 0x000001A5D43F9CF0>), ('marketCap', 1977535730.11456, <function gt at 0x000001A5D43F9CF0>)), 491)


(6.803150219406607, (('fullTimeEmployees', 7686.0, <function gt at 0x000001A5D43F9CF0>), ('country', 'Japan', <function eq at 0x000001A5D43F9E10>)), 616)


(6.650918653291399, (('country', 'Japan', <function eq at 0x000001A5D43F9E10>), ('marketCap', 12953903104.0, <function leeq at 0x000001A5D43F9D80>), ('fullTimeEmployees', 1817.0, <function gt at 0x000001A5D43F9CF0>)), 575)


(6.205676446380705, (('country', 'Japan', <function eq at 0x000001A5D43F9E10>), ('enterpriseToEbitda', -1.044, <function gt at 0x000001A5D43F9CF0>)), 656)


(5.8773758485413, (('fullTimeEmployees', 46000.0, <function gt at 0x000001A5D43F9CF0>), ('enterpriseToEbitda', 9.382, <function leeq at 0x000001A5D43F9D80>), ('enterpriseToEbitda', 8.486, <function leeq at 0x000001A5D43F9D80>)), 535)



In [7]:
descriptors = results[0][1]
df = filter_df_on_descriptors(stock_df_index, descriptors)
df

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,fullTimeEmployees,growth_target
13,Japan,Auto Manufacturers,EUR,Europe/Berlin,FRA,Consumer Cyclical,0.0,1.347,8.373396e+09,57.726,51249.0,"[0.0, -0.3, 0.13, -2.27, -7.6, 6.83, -5.39, -1..."
79,Japan,Auto Parts,EUR,Europe/Berlin,FRA,Consumer Cyclical,0.0,0.328,2.590374e+10,25.747,135636.0,"[0.0, 0.85, 0.8, -4.17, -5.86, 0.46, -1.03, -3..."
84,Japan,Marine Shipping,EUR,Europe/Berlin,FRA,Industrials,46.0,1.864,3.703204e+09,42.062,35165.0,"[0.0, 3.86, 1.9, -13.63, -12.04, 11.3, 0.12, -..."
148,Japan,Electrical Equipment & Parts,EUR,Europe/Berlin,FRA,Industrials,2.0,0.616,5.795071e+09,24.438,26757.0,"[0.0, 5.96, 5.33, -5.67, -8.94, 3.39, 3.97, 7...."
159,Japan,Consumer Electronics,EUR,Europe/Berlin,FRA,Technology,2190.0,0.961,2.100761e+10,43.900,241423.0,"[0.0, -2.49, -3.51, 7.56, -10.28, 5.86, -5.7, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9645,Japan,Auto Manufacturers,ARS,America/Argentina/Buenos_Aires,BUE,Consumer Cyclical,145.0,8.486,3.974523e+12,102.678,376971.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 97.3,..."
9774,Japan,Electrical Equipment & Parts,JPY,Europe/London,LSE,Industrials,4893.0,6.287,2.079737e+10,10.654,145696.0,"[0.0, 1.08, 10.44, -11.75, -11.15, 7.96, -9.17..."
9775,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,78360.0,"[0.0, 5.82, 1.71, 7.84, -10.55, 3.58, -7.94, 2..."
9776,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,4894.0,"[0.0, 6.5, 0.0, -10.85, -1.95, -3.62, -3.95, 9..."
